# Grounding with Bing Search and Bing Custom Search

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import the libraries

In [25]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool
#from azure.ai.agents.models import BingCustomSearchTool

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [26]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Create the Bing tools

In [27]:
# Enter Bing connection names here
bing_search = "bingsearch"  # Replace with your Bing connection name
bing_custom_search = "bingcustomsearch"  # Replace with your Bing Custom Search connection name
bing_custom_configuration = "bingcustomsearch"  # Replace with your Bing Custom Configuration name

# Extract the connection list.
conn_list = project_client.connections.list()
bing_search_conn_id = ""
bing_custom_search_conn_id = ""

# Find the connection IDs for Bing Search and Bing Custom Search
for conn in conn_list:
    if conn.name == bing_search:
        bing_search_conn_id = conn.id
    elif conn.name == bing_custom_search:
        bing_custom_search_conn_id = conn.id

# Print the connection names
print(f"Bing Search Connection ID: {bing_search_conn_id}")
print(f"Bing Custom Search Connection ID: {bing_custom_search_conn_id}")

Bing Search Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/agent-standard/providers/Microsoft.CognitiveServices/accounts/foundrydlt4/projects/projectdlt4/connections/bingsearch
Bing Custom Search Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/agent-standard/providers/Microsoft.CognitiveServices/accounts/foundrydlt4/projects/projectdlt4/connections/bingcustomsearch


## Create the agent

In [28]:
# Initialize the Bing Grounding tools
bing_search = BingGroundingTool(connection_id=bing_search_conn_id)
#bing_custom_search = BingCustomSearchTool(connection_id=bing_custom_search, instance_name=bing_custom_configuration)

# Create an agent with the Bing Grounding tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-bing-agent",  # Name of the agent
    instructions="You are a helpful agent",  # Instructions for the agent
    tools=bing_search.definitions,  # Attach the Bing Grounding tool
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_UfYFHLjClfPMejzHiyS1cOZ3


## Create thread

In [29]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_72Z5ePbQtHg2JSgRedq5ANJC


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [30]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

     # Fetch run steps to get the details of the agent run
    run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                bing_grounding_details = call.get("bing_grounding", {})
                if bing_grounding_details:
                    print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

        print()  # add an extra newline between steps

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        # Fetch the first message in the thread to see the agent's response
        messages = list(project_client.agents.messages.list(thread_id=thread.id))
        if messages:
            first_message = messages[0]
            content = first_message['content']
            # Extract the text value if present
            if isinstance(content, list) and content:
                text_value = content[0].get('text', {}).get('value', '')
                print(text_value)


## Run the agent

In [32]:
user_input = "What is the weather in Seattle?"
run_agent(user_input)

Created message, ID: msg_zsXDizGukAucNSeaa1Z429f6
Run finished with status: completed
Currently, the weather in Seattle is mostly cloudy with a temperature of around 63°F (17°C). The wind is blowing from the north-northeast at about 3 mph. The forecast for today indicates a high of 76°F (24°C) and a low of 55°F (13°C), with conditions becoming cloudy later in the day【3:0†source】.


## Cleanup resources

In [33]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
